## Lexical Analyzer

Step 1: Define Token Patterns
First, define the set of tokens that your language supports along with their corresponding regular expressions. For simplicity, let’s consider a small language that includes identifiers, integers, parentheses, and basic arithmetic operators.

In [ ]:
import re

# Define the token categories and their regex patterns
TOKEN_PATTERNS = {
    'INTEGER': r'\d+',
    'IDENTIFIER': r'[a-zA-Z_][a-zA-Z_0-9]*',
    'OPERATOR': r'[\+\-\*/]',
    'LPAREN': r'\(',
    'RPAREN': r'\)',
    'WHITESPACE': r'\s+',
}


Step 2: Build the Lexer
The lexer reads the input text, matches the text against the defined patterns, and produces a list of tokens.

In [ ]:
class Token:
    def __init__(self, type, value):
        self.type = type
        self.value = value

    def __str__(self):
        return f'Token({self.type}, {self.value})'

class Lexer:
    def __init__(self, patterns):
        # Compile the string patterns into regex objects for faster matching
        self.patterns = {key: re.compile(pattern) for key, pattern in patterns.items()}

    def tokenize(self, text):
        pos = 0
        tokens = []

        while pos < len(text):
            match = None
            for type, pattern in self.patterns.items():
                match = pattern.match(text, pos)
                if match:
                    # Skip whitespace
                    if type != 'WHITESPACE':
                        value = match.group(0)
                        tokens.append(Token(type, value))
                    pos = match.end()
                    break
            if not match:
                raise SyntaxError(f'Illegal character at index {pos}')
        return tokens


Step 3: Test the Lexer
Now, test the lexer with a simple input string to see if it correctly identifies the tokens.

In [ ]:
if __name__ == '__main__':
    input_text = "(sum + 47) / total"
    lexer = Lexer(TOKEN_PATTERNS)
    tokens = lexer.tokenize(input_text)

    for token in tokens:
        print(token)


Token(LPAREN, ()
Token(IDENTIFIER, sum)
Token(OPERATOR, +)
Token(INTEGER, 47)
Token(RPAREN, ))
Token(OPERATOR, /)
Token(IDENTIFIER, total)


Explanation
Define Token Patterns: Regular expressions for each token type are defined.
Token and Lexer Classes: A Token class to represent individual tokens and a Lexer class to tokenize the input string.
The Lexer class’s tokenize method processes the input text, matches each part of the string to the defined token patterns, and generates a list of Token objects.
Skipping Whitespace: Whitespace tokens are recognized but not added to the final list of tokens.
Error Handling: If the lexer encounters an unrecognized character, it raises a SyntaxError.

---



---



## Parsers (Syntax Analysis): Run on Spyder/Pycharm etc

Step 1: Install PLY
Ensure you have PLY installed in your environment:

In [ ]:
pip install ply


Step 2: Define the Lexer
The lexer breaks the input text into a stream of tokens. We define tokens for numbers, addition and multiplication operators, parentheses, and whitespace.

In [ ]:
import ply.lex as lex

# List of token names.
tokens = (
    'NUMBER',
    'PLUS',
    'TIMES',
    'LPAREN',
    'RPAREN',
)

# Regular expression rules for simple tokens
t_PLUS = r'\+'
t_TIMES = r'\*'
t_LPAREN = r'\('
t_RPAREN = r'\)'

# A regular expression rule with some action code
def t_NUMBER(t):
    r'\d+'
    t.value = int(t.value)    # Convert string to integer
    return t

# Define a rule so we can track line numbers
def t_newline(t):
    r'\n+'
    t.lexer.lineno += len(t.value)

# A string containing ignored characters (spaces and tabs)
t_ignore  = ' \t'

# Error handling rule
def t_error(t):
    print(f"Illegal character '{t.value[0]}'")
    t.lexer.skip(1)

# Build the lexer
lexer = lex.lex()



TypeError: <module '__main__'> is a built-in module

Step 3: Define the Parser
The parser analyzes the token stream to determine its grammatical structure with respect to the given grammar.

In [ ]:
import ply.yacc as yacc

# Define the precedence and associativity of operators
precedence = (
    ('left', 'PLUS'),
    ('left', 'TIMES'),
)

# The grammar rules and handler functions
def p_expression_plus(p):
    'expression : expression PLUS term'
    p[0] = p[1] + p[3]  # Perform addition

def p_expression_times(p):
    'expression : term TIMES factor'
    p[0] = p[1] * p[3]  # Perform multiplication

def p_expression_term(p):
    'expression : term'
    p[0] = p[1]

def p_term_factor(p):
    'term : factor'
    p[0] = p[1]

def p_factor_number(p):
    'factor : NUMBER'
    p[0] = p[1]

def p_factor_expr(p):
    'factor : LPAREN expression RPAREN'
    p[0] = p[2]  # Parentheses are for grouping, so just return the inner expression value

# Error rule for syntax errors
def p_error(p):
    print("Syntax error in input!")

# Build the parser
parser = yacc.yacc()

Step 4: Using the Parser
Now, you can use the lexer and parser to evaluate expressions.

In [ ]:
# Test it out
data = "3 * 4 + 5"

# Give the lexer some input
lexer.input(data)
# Iterate through tokens
for token in lexer:
    print(token)
# Parse the input string
result = parser.parse(data)
print(result)

Explanation:
Tokens: We define tokens for numbers, plus, times, left parenthesis, and right parenthesis.
Lexer Rules: Each token type has an associated regular expression. For numbers, we also convert the matched string to an integer.
Parser Rules: The parser rules define how tokens can be combined to form valid expressions. We use Python functions for each grammar rule. The p[0] = p[1] + p[3] syntax is used to calculate the result of binary operations, where p[1] and p[3] refer to the operands and p[2] is the operator (ignored in calculation).
Precedence: The precedence rules ensure that multiplication is evaluated before addition.
Error Handling: Basic error handling is provided to skip illegal characters in the lexer and to print a message in case of syntax errors in the parser.

---


---




## Semantic Analysis (Manual)

Step 1: Define Symbol Table
A symbol table is needed to store information about variables (like their types and scopes). Here's a basic implementation:

In [ ]:
class Symbol:
    def __init__(self, name, type=None):
        self.name = name
        self.type = type

class SymbolTable:
    def __init__(self, parent=None):
        self.symbols = {}
        self.parent = parent

    def add(self, symbol):
        self.symbols[symbol.name] = symbol

    def get(self, name):
        symbol = self.symbols.get(name, None)
        if symbol is not None:
            return symbol
        if self.parent is not None:
            return self.parent.get(name)
        return None


Step 2: Enhance the Parser for Semantic Analysis
Extend the parser to perform semantic analysis, such as type checking and symbol table management. This example builds upon a hypothetical parser that produces an AST.

In [ ]:
class Node:
    pass

class BinOp(Node):
    def __init__(self, left, op, right):
        self.left = left
        self.op = op
        self.right = right

class Num(Node):
    def __init__(self, value):
        self.value = value
        self.type = 'int' if isinstance(value, int) else 'float'

class Var(Node):
    def __init__(self, name):
        self.name = name
        self.type = None  # To be filled in by the semantic analyzer

# Assume `ast` is the root of the AST produced by the parser


Step 3: Implement the Semantic Analyzer
The semantic analyzer walks the AST, updates the symbol table, and performs type checking.

In [ ]:
class SemanticAnalyzer:
    def __init__(self):
        self.global_scope = SymbolTable()

    def visit(self, node, table):
        method_name = f'visit_{type(node).__name__}'
        visitor = getattr(self, method_name, self.generic_visit)
        return visitor(node, table)

    def visit_BinOp(self, node, table):
        left_type = self.visit(node.left, table)
        right_type = self.visit(node.right, table)
        if left_type != right_type:
            raise TypeError("Type mismatch in binary operation")
        return left_type  # Assuming the operation doesn't change the type

    def visit_Num(self, node, table):
        return node.type

    def visit_Var(self, node, table):
        symbol = table.get(node.name)
        if symbol is None:
            raise NameError(f"Variable '{node.name}' not defined")
        node.type = symbol.type
        return node.type

    def generic_visit(self, node, table):
        raise Exception(f"No visit_{type(node).__name__} method")

    def analyze(self, ast):
        self.visit(ast, self.global_scope)


Step 4: Use the Semantic Analyzer
After parsing the AST, pass it to the semantic analyzer for analysis.

In [ ]:
# Assuming `ast` is the root node of the AST
analyzer = SemanticAnalyzer()
try:
    analyzer.analyze(ast)
    print("Semantic analysis completed successfully.")
except (TypeError, NameError) as e:
    print(f"Semantic error: {e}")


Semantic error: Variable 'a' not defined


To demonstrate different outputs from the semantic analyzer based on various inputs, let's consider a few scenarios in our hypothetical language environment. These scenarios will illustrate successful semantic analysis and common semantic errors such as type mismatches and undeclared variables.

Prerequisites
Assuming the semantic analyzer and all necessary classes (SymbolTable, Symbol, Node, BinOp, Num, Var, etc.) have been defined as previously described, we'll create some example ASTs that represent different expressions or statements in the language.

Scenario 1: Successful Semantic Analysis
Input Code: 3 + 4

This code should pass semantic analysis since both operands are integers, and the operation is valid.

In [ ]:
# AST for "3 + 4"
ast = BinOp(Num(3), '+', Num(4))

# Semantic analysis
analyzer = SemanticAnalyzer()
try:
    analyzer.analyze(ast)
    print("Semantic analysis completed successfully.")
except Exception as e:
    print(f"Semantic error: {e}")


Semantic analysis completed successfully.


Scenario 2: Type Mismatch Error
Input Code: 3 + 4.5

This code should raise a type mismatch error during semantic analysis because the operands are of different types.

In [ ]:
# AST for "3 + 4.5"
ast = BinOp(Num(3), '+', Num(4.5))

# Semantic analysis
analyzer = SemanticAnalyzer()
try:
    analyzer.analyze(ast)
    print("Semantic analysis completed successfully.")
except Exception as e:
    print(f"Semantic error: {e}")


Semantic error: Type mismatch in binary operation


Scenario 3: Undeclared Variable Error
Input Code: a + 5 (assuming a has not been declared)

This code should raise an undeclared variable error since a has not been added to the symbol table.

In [ ]:
# AST for "a + 5", assuming 'a' is not declared
ast = BinOp(Var('a'), '+', Num(5))

# Semantic analysis
analyzer = SemanticAnalyzer()
try:
    analyzer.analyze(ast)
    print("Semantic analysis completed successfully.")
except Exception as e:
    print(f"Semantic error: {e}")


Semantic error: Variable 'a' not defined


## Complete compiler

Step 1: Install PLY
Ensure PLY is installed in your environment:

In [ ]:
pip install ply


Step 2: Define the Lexer
First, define the lexer using PLY's lexing capabilities. This lexer recognizes tokens for identifiers, numbers, assignment operators, arithmetic operators, and parentheses.

In [ ]:
import ply.lex as lex

tokens = ('NUMBER', 'ID', 'ASSIGN', 'PLUS', 'TIMES', 'LPAREN', 'RPAREN')

t_ASSIGN = r'='
t_PLUS = r'\+'
t_TIMES = r'\*'
t_LPAREN = r'\('
t_RPAREN = r'\)'

def t_ID(t):
    r'[a-zA-Z_][a-zA-Z_0-9]*'
    return t

def t_NUMBER(t):
    r'\d+'
    t.value = int(t.value)
    return t

t_ignore = ' \t\n'

def t_error(t):
    print(f"Illegal character '{t.value[0]}'")
    t.lexer.skip(1)

lexer = lex.lex()


TypeError: <module '__main__'> is a built-in module

Step 3: Define the Symbol Table
Implement a simple symbol table to track variable declarations.

In [ ]:
class SymbolTable:
    def __init__(self):
        self.table = {}

    def define(self, name, type):
        self.table[name] = type

    def lookup(self, name):
        return self.table.get(name, None)


Step 4: Define the Parser and Integrate Semantic Analysis
Define the parser using PLY's yacc module, integrating semantic checks using the symbol table.

In [ ]:
import ply.yacc as yacc

# Assuming the lexer tokens are defined as above

precedence = (
    ('left', 'PLUS'),
    ('left', 'TIMES'),
)

# Symbol table for semantic analysis
symtable = SymbolTable()

def p_statement_assign(p):
    'statement : ID ASSIGN expression'
    # Semantic check: Ensure variable is defined
    if symtable.lookup(p[1]) is None:
        print(f"Semantic error: Undefined variable '{p[1]}' at assignment")
    else:
        # Here, we simply print the assignment for demonstration
        print(f"Assigning: {p[1]} = {p[3]}")

def p_statement_decl(p):
    'statement : ID'
    # Assume every ID token is a variable declaration for simplicity
    symtable.define(p[1], 'int')  # Simulate integer type
    print(f"Declared variable: {p[1]}")

def p_expression_binop(p):
    '''expression : expression PLUS expression
                  | expression TIMES expression'''
    if p[2] == '+':
        p[0] = p[1] + p[3]
    elif p[2] == '*':
        p[0] = p[1] * p[3]

def p_expression_group(p):
    'expression : LPAREN expression RPAREN'
    p[0] = p[2]

def p_expression_number(p):
    'expression : NUMBER'
    p[0] = p[1]

def p_expression_id(p):
    'expression : ID'
    # Semantic check: Ensure variable is defined
    if symtable.lookup(p[1]) is None:
        print(f"Semantic error: Undefined variable '{p[1]}'")
    else:
        # Placeholder for a variable's value
        p[0] = f"value_of({p[1]})"

def p_error(p):
    print("Syntax error in input!")

parser = yacc.yacc()


Step 5: Test the Compiler Components
Now, test the lexer, parser, and semantic analysis with a simple input.

In [ ]:
test_input = """
x = 3
y = x + 4
z = y * 2
"""

# Test Lexer
print("Lexing result:")
lexer.input(test_input)
for tok in lexer:
    print(tok)

# Test Parser and Semantic Analysis
print("\nParsing and Semantic Analysis result:")
parser.parse(test_input)


Expected Output
The lexing step should output the tokens found in the test_input. Then, the parsing and semantic analysis steps should print messages about variable declarations, assignments, and any semantic errors detected (such as using an undefined variable).

MINI Compiler All Steps

In [1]:
import re

# Token specification
TOKEN_SPECIFICATION = [
    ('NUMBER',   r'\d+'),
    ('OPERATOR', r'[+\-*/]'),
    ('LPAREN',   r'\('),
    ('RPAREN',   r'\)'),
    ('WS',       r'\s+'),
]

# Tokenize function
def tokenize(expression):
    token_regex = '|'.join(f'(?P<{name}>{pattern})' for name, pattern, *_ in TOKEN_SPECIFICATION)
    for match in re.finditer(token_regex, expression):
        kind = match.lastgroup
        value = match.group()
        if kind != 'WS':  # Skip whitespace
            yield (kind, value)



# AST Node Classes
class ASTNode: pass

class NumberNode(ASTNode):
    def __init__(self, value):
        self.value = value
    def __repr__(self):
        return f"NumberNode({self.value})"

class BinOpNode(ASTNode):
    def __init__(self, left, op, right):
        self.left = left
        self.op = op
        self.right = right
    def __repr__(self):
        return f"BinOpNode({repr(self.left)}, '{self.op}', {repr(self.right)})"


# Operator precedence
operator_precedence = {
    '+': 15,
    '-': 10,
    '*': 20,
    '/': 30,
}







# Parse function
def parse(tokens):
    tokens = list(tokens)  # Convert generator to list
    ast, _ = parse_expr(tokens, 0)
    return ast

# Parse expressions with precedence handling
def parse_expr(tokens, min_precedence):
    lhs, tokens = parse_primary(tokens)  # Initial primary expression
    while tokens and tokens[0][0] == 'OPERATOR':
        op = tokens[0][1]
        prec = operator_precedence[op]
        if prec >= min_precedence:
            # Consume the operator
            tokens.pop(0)
            # Parse the right-hand side of the operator
            rhs, tokens = parse_primary(tokens)
            # Recursively call parse_expr to handle the rest of the expression
            # This allows handling right-associativity and precedence correctly
            while tokens and tokens[0][0] == 'OPERATOR' and operator_precedence[tokens[0][1]] > prec:
                rhs, tokens = parse_expr(tokens, operator_precedence[tokens[0][1]])
            lhs = BinOpNode(lhs, op, rhs)
        else:
            break
    return lhs, tokens



# Parse primary elements (numbers and expressions in parentheses)
def parse_primary(tokens):
    token = tokens.pop(0)
    if token[0] == 'NUMBER':
        return NumberNode(token[1]), tokens
    elif token[1] == '(':
        # Recursively parse the expression inside the parentheses
        expr, tokens = parse_expr(tokens, 0)
        # Expect and consume the closing parenthesis
        if tokens and tokens[0][1] == ')':
            tokens.pop(0)  # Remove the ')'
            return expr, tokens
        else:
            raise SyntaxError("Expected ')'")
    else:
        raise SyntaxError(f"Unexpected token: {token[1]}")





# Code generation (to Python source code)
def generate_python_code(node):
    if isinstance(node, NumberNode):
        return node.value
    elif isinstance(node, BinOpNode):
        left_code = generate_python_code(node.left)
        right_code = generate_python_code(node.right)
        return f"({left_code} {node.op} {right_code})"






# Compile expression to Python source code
def compile_expression_to_python_source(expression):
    tokens = tokenize(expression)
    ast = parse(tokens)
    python_code = generate_python_code(ast)
    return python_code






#outputs
#Step 1: Display Tokenization Output
def tokenize(expression):
    token_regex = '|'.join(f'(?P<{name}>{pattern})' for name, pattern, *_ in TOKEN_SPECIFICATION)
    tokens = []
    for match in re.finditer(token_regex, expression):
        kind = match.lastgroup
        value = match.group()
        if kind != 'WS':  # Skip whitespace
            tokens.append((kind, value))
    print("Tokens:", tokens)
    return tokens

#Step 2: Display Parsing Output (AST Structure)
def parse(tokens):
    tokens = list(tokens)  # Convert generator to list
    ast, _ = parse_expr(tokens, 0)
    print("AST Structure:", ast)
    return ast

#Print
expression = "3 + (5 * 2)-2"
python_code = compile_expression_to_python_source(expression)
print("Generated Python Source Code:", python_code)
result = eval(python_code)
print("Result of the expression:", result)



Tokens: [('NUMBER', '3'), ('OPERATOR', '+'), ('LPAREN', '('), ('NUMBER', '5'), ('OPERATOR', '*'), ('NUMBER', '2'), ('RPAREN', ')'), ('OPERATOR', '-'), ('NUMBER', '2')]
AST Structure: BinOpNode(BinOpNode(NumberNode(3), '+', BinOpNode(NumberNode(5), '*', NumberNode(2))), '-', NumberNode(2))
Generated Python Source Code: ((3 + (5 * 2)) - 2)
Result of the expression: 11
